In [1]:
import sys
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [ ]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
!pip install transformers[sentencepiece] datasets spacy scipy networkx numpy sent2vec pyngrok pandas nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=d8ffc81f8ba552af02844068d010894f32ac55717426039f3a99370d2fec1b79
  Sto

In [ ]:
!pip install rouge_score evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=18fcac486c47dc0c2781810c5ad6ce800945f1d0e9b11233a5b670036c0b3c8b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install openpyxl matplotlib

In [ ]:
# import library yang dibutuhkan
import spacy
import networkx as nx
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

import pandas as pd
from rouge_score import rouge_scorer
from openpyxl import Workbook
import matplotlib.pyplot as plt

from scipy import spatial
from sent2vec.vectorizer import Vectorizer

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import TFPegasusForConditionalGeneration, PegasusTokenizerFast

import re
import unicodedata

**PERINGKASAN BAHASA INGGRIS**

In [ ]:
# 1. FUNGSI PRE-PROCESSING
def preprocess_en(text):
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(text)

  # Segmentasi (pisahkan text per kalimat dan masukkan kedalam list)
  sentences = [sent.text.strip() for sent in doc.sents]

  # Membersihkan setiap kalimat
  filtered_sentences = []
  for i in range(len(sentences)):
    doc = nlp(sentences[i])

    # inisial variabel untuk menampung kata yang bersih
    filtered_tokens = []

    # tokenisasi
    for token in doc:
    # menghilangkan stopword dan karakter yang tidak dibutuhkan
      if token.is_stop or token.is_punct:
        continue
      # lemmatisasi
      filtered_tokens.append(token.lemma_)

    # how to convert list to string
    result = " ".join(filtered_tokens)

    filtered_sentences.append(result)
  return filtered_sentences

In [ ]:
# Load the tokenizer and model
tokenizer_bert_en = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert_en = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
def vector_en(list_sentences):

  # Initialize an empty array to store sentence embeddings
  sentence_embeddings = []

  # Iterate through sentences and get embeddings
  for text in list_sentences:
      input_ids = tokenizer_bert_en.encode(text, add_special_tokens=True, padding=True, truncation=True)
      input_ids = torch.tensor(input_ids).unsqueeze(0)

      with torch.no_grad():
          outputs = model_bert_en(input_ids)
          cls_embedding = outputs.last_hidden_state[:, 0, :]
          embedding = cls_embedding[0]

      sentence_embeddings.append(embedding.numpy())

  return sentence_embeddings

In [ ]:
# 2. FUNGSI PERINGKASAN EKSTRAKTIF
def extractive_sum_en(filtered_sentences, sentences, n):
  # mengubah kalimat menjadi vektor
  vectors = vector_en(filtered_sentences)

  # menghitung similarity matrix (matriks kemiripan antar kalimat)
  similarity_matrix = []
  for i in range(len(vectors)):
    row = []
    for j in range(len(vectors)):
      row.append(spatial.distance.cosine(vectors[i], vectors[j]))
    similarity_matrix.append(row)

  # konversi matrix menjadi graph
  graph = nx.from_numpy_array(np.array(similarity_matrix))

  # melakukan perangkingan
  scores = nx.pagerank(graph)

  # mengambil top rank (kalimat dengan score tertinggi)
  sentences_size = len(sentences)
  num_sentences = round((sentences_size + 1) / n)

  top_sentence_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:num_sentences]
  summary = [sentences[i] for i in top_sentence_indices]
  summary = " ".join(summary)

  # hasil peringkasan ekstraktif
  return summary


In [ ]:
# 3. FUNGSI PERINGKASAN ABSTRAKTIF

model_name1 = "google/pegasus-cnn_dailymail"
model1 = PegasusForConditionalGeneration.from_pretrained(model_name1)
tokenizer1 = PegasusTokenizer.from_pretrained(model_name1)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def abstractive_sum_en(text):

  inputs = tokenizer1.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)

  summary_ids = model1.generate(inputs, max_length=350, min_length=50, length_penalty=2.0, num_beams=6, early_stopping=True)

  summary = tokenizer1.decode(summary_ids[0], skip_special_tokens=True)

  return summary

In [ ]:
# MERINGKAS TEKS BAHASA INGGRIS
def summy_en(text, n):
  document = text
  # load spacy
  nlp = spacy.load("en_core_web_sm")

  # memasukkan kalimat kedalam list (untuk memudahkan print output)
  doc = nlp(document)
  sentences = [sent.text.strip() for sent in doc.sents]

  # melakukan pre-processing
  # contoh hasil penggunaan fungsi preprocess
  result = preprocess_en(document)

  # melakukan peringkasan ekstraktif
  summary1 = extractive_sum_en(result, sentences, n)

  # melakukan peringkasan abstraktif
  summary = abstractive_sum_en(summary1)
  summary = summary.replace("<pad>", "")
  summary = summary.replace("</s>", "")
  summary = summary.replace(".<n>", ".\n")

  return summary


In [ ]:
text = """

Python is a high-level, interpreted programming language that is widely used for web development, data analysis, artificial intelligence, and many other applications. Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages in the world, thanks to its simplicity, versatility, and wide range of libraries and frameworks.

Python is known for its elegant syntax, which emphasizes readability and reduces the cost of program maintenance. Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. This makes Python code more concise and easier to read, especially for beginners who are just learning how to program.

In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making it a versatile tool for different kinds of applications. Python also has a large standard library and a vast ecosystem of third-party libraries and packages, which makes it easy to find and use pre-built modules for different purposes.

Whether you are a beginner or an experienced programmer, learning Python can open up many opportunities for you. With its growing popularity and wide range of applications, Python is an essential skill for many industries and fields. So why not start learning Python today and see where it takes you?

"""

In [ ]:
result_en = summy_en(text, 2)

In [ ]:
print(result_en)

Python supports object-oriented programming, functional programming, and procedural programming paradigms.
Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation.
 Python is an essential skill for many industries and fields.


**PERINGKASAN BAHASA INGGRIS PENGUJIAN**

In [ ]:
# 1. TEXTRANK
def f_textrank_en(text, n):
  document = text
  nlp = spacy.load("en_core_web_sm")

  # memasukkan kalimat kedalam list (untuk memudahkan print output)
  doc = nlp(document)
  sentences = [sent.text.strip() for sent in doc.sents]

  # melakukan pre-processing
  # contoh hasil penggunaan fungsi preprocess
  result = preprocess_en(document)

  # melakukan peringkasan ekstraktif
  summary = extractive_sum_en(result, sentences, n)

  return summary

In [ ]:
# 2. PEGASUS
def f_pegasus_en(text):
  summary = abstractive_sum_en(text)
  return summary

In [ ]:
# 3. HYBRID
def f_hybrid_en(text, n):
  summary = summy_en(text, n)
  return summary

In [ ]:
text1 = """

Python is a high-level, interpreted programming language that is widely used for web development, data analysis, artificial intelligence, and many other applications. Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages in the world, thanks to its simplicity, versatility, and wide range of libraries and frameworks.

Python is known for its elegant syntax, which emphasizes readability and reduces the cost of program maintenance. Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. This makes Python code more concise and easier to read, especially for beginners who are just learning how to program.

In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making it a versatile tool for different kinds of applications. Python also has a large standard library and a vast ecosystem of third-party libraries and packages, which makes it easy to find and use pre-built modules for different purposes.

Whether you are a beginner or an experienced programmer, learning Python can open up many opportunities for you. With its growing popularity and wide range of applications, Python is an essential skill for many industries and fields. So why not start learning Python today and see where it takes you?

"""

In [ ]:
hasil_textrank_en = f_textrank_en(text1, 3)
hasil_pegasus_en = f_pegasus_en(text1)
hasil_hybrid_en = f_hybrid_en(text1, 2)

In [ ]:
hasil_textrank_en

'Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. So why not start learning Python today and see where it takes you? Whether you are a beginner or an experienced programmer, learning Python can open up many opportunities for you. With its growing popularity and wide range of applications, Python is an essential skill for many industries and fields.'

In [ ]:
hasil_pegasus_en

'Python is a high-level, interpreted programming language.<n>It is widely used for web development, data analysis, artificial intelligence, and many other applications.<n>Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages.'

In [ ]:
hasil_hybrid_en

'Python supports object-oriented programming, functional programming, and procedural programming paradigms.\nUnlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation.\n Python is an essential skill for many industries and fields.'

**PENGUJIAN PADA MASING MASING DATASET**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/NLP/f_zero/hasil_uji

/content/gdrive/MyDrive/NLP/f_zero/hasil_uji


PENGUJIAN PADA 50 ARTIKEL CNN/DAILY MAIL

In [ ]:
import evaluate

In [ ]:
def save_test_data(file_name, reference_summary, generated_summary):
  # Inisialisasi ROUGE scorer
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

  # Menghitung skor ROUGE untuk setiap pasangan referensi dan hasil yang dihasilkan
  scores = []
  for ref, gen in zip(reference_summary, generated_summary):
      score = scorer.score(ref, gen)
      scores.append(score)

  # Membuat DataFrame dari skor ROUGE, precision, dan recall
  data = {
      'Reference': reference_summary,
      'Generated': generated_summary,
      'ROUGE-1 Precision': [score['rouge1'].precision for score in scores],
      'ROUGE-1 Recall': [score['rouge1'].recall for score in scores],
      'ROUGE-1 F1-score': [score['rouge1'].fmeasure for score in scores],
      'ROUGE-2 Precision': [score['rouge2'].precision for score in scores],
      'ROUGE-2 Recall': [score['rouge2'].recall for score in scores],
      'ROUGE-2 F1-score': [score['rouge2'].fmeasure for score in scores],
      'ROUGE-L Precision': [score['rougeL'].precision for score in scores],
      'ROUGE-L Recall': [score['rougeL'].recall for score in scores],
      'ROUGE-L F1-score': [score['rougeL'].fmeasure for score in scores],
  }

  df = pd.DataFrame(data)

  # Menyimpan DataFrame ke file Excel
  output_excel = file_name
  df.to_excel(output_excel, index=False, engine='openpyxl')

  return(f"DataFrame saved to {output_excel}")


In [ ]:
# import and load dataset
from datasets import load_dataset

dataset_cnn = load_dataset("cnn_dailymail", version="3.0.0")

print(f"Features in cnn_dailymail : {dataset_cnn['train'].column_names}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Features in cnn_dailymail : ['article', 'highlights', 'id']


In [ ]:
summary_textrank_cnn = []
summary_pegasus_cnn = []
summary_hybrid_cnn = []
reference_summary_cnn = []

limit = 50

In [ ]:
for i in range (limit):
  # Inisial reference
  reference = dataset_cnn["test"][i]["highlights"]
  reference_summary_cnn.append(reference)

In [ ]:
# textrank rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_cnn["test"][i]["article"]
  result = f_textrank_en(input, 2)

  summary_textrank_cnn.append(result)

file_name = 'pengujian_textrank_cnn.xlsx'

print_test = save_test_data(file_name, reference_summary_cnn, summary_textrank_cnn)

In [ ]:
# # pegasus rouge score
# for i in range (limit):
#   # Inisial kalimat inputan
#   input = dataset_cnn["test"][i]["article"]
#   result = f_pegasus_en(input)

#   summary_pegasus_cnn.append(result)

# file_name = 'pengujian_pegasus_cnn.xlsx'

# print_test = save_test_data(file_name, reference_summary_cnn, summary_pegasus_cnn)

In [ ]:
# # hybrid rouge score
# for i in range (limit):
#   # Inisial kalimat inputan
#   input = dataset_cnn["test"][i]["article"]
#   result = f_hybrid_en(input, 2)

#   summary_hybrid_cnn.append(result)

# file_name = 'pengujian_hybrid_cnn.xlsx'

# print_test = save_test_data(file_name, reference_summary_cnn, summary_hybrid_cnn)

PENGUJIAN PADA 50 ARTIKEL PUBMED

In [ ]:
# import and load dataset
from datasets import load_dataset

dataset_pubmed = load_dataset("ccdv/pubmed-summarization")

print(f"Features in pubmed : {dataset_pubmed['train'].column_names}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Features in pubmed : ['article', 'abstract']


In [ ]:
summary_textrank_pubmed = []
summary_pegasus_pubmed = []
summary_hybrid_pubmed = []
reference_summary_pubmed = []

limit = 50

In [ ]:
for i in range (limit):
  # Inisial reference
  reference = dataset_pubmed["test"][i]["abstract"]
  reference_summary_pubmed.append(reference)

In [ ]:
# textrank rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_pubmed["test"][i]["article"]
  result = f_textrank_en(input, 2)

  summary_textrank_pubmed.append(result)

file_name = 'pengujian_textrank_pubmed.xlsx'

print_test = save_test_data(file_name, reference_summary_pubmed, summary_textrank_pubmed)
print_test

'DataFrame saved to pengujian_textrank_pubmed.xlsx'

In [ ]:
# pegasus rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_pubmed["test"][i]["article"]
  result = f_pegasus_en(input)

  summary_pegasus_pubmed.append(result)

file_name = 'pengujian_pegasus_pubmed.xlsx'

print_test = save_test_data(file_name, reference_summary_pubmed, summary_pegasus_pubmed)
print_test

'DataFrame saved to pengujian_pegasus_pubmed.xlsx'

In [ ]:
# hybrid rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_pubmed["test"][i]["article"]
  result = f_hybrid_en(input, 2)

  summary_hybrid_pubmed.append(result)

file_name = 'pengujian_hybrid_pubmed.xlsx'

print_test = save_test_data(file_name, reference_summary_pubmed, summary_hybrid_pubmed)
print_test

'DataFrame saved to pengujian_hybrid_pubmed.xlsx'

PENGUJIAN PADA 50 ARTIKEL MULTINEWS

In [ ]:
# import and load dataset
from datasets import load_dataset

dataset_multi = load_dataset("multi_news")

print(f"Features in multi : {dataset_multi['train'].column_names}")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Features in multi : ['document', 'summary']


In [ ]:
summary_textrank_multi = []
summary_pegasus_multi = []
summary_hybrid_multi = []
reference_summary_multi = []

limit = 50

In [ ]:
for i in range (limit):
  # Inisial reference
  reference = dataset_multi["test"][i]["summary"]
  reference_summary_multi.append(reference)

In [ ]:
# textrank rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_multi["test"][i]["document"]
  result = f_textrank_en(input, 2)

  summary_textrank_multi.append(result)

file_name = 'pengujian_textrank_multi.xlsx'

print_test = save_test_data(file_name, reference_summary_multi, summary_textrank_multi)
print_test

'DataFrame saved to pengujian_textrank_multi.xlsx'

In [ ]:
# pegasus rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_multi["test"][i]["document"]
  result = f_pegasus_en(input)

  summary_pegasus_multi.append(result)

file_name = 'pengujian_pegasus_multi.xlsx'

print_test = save_test_data(file_name, reference_summary_multi, summary_pegasus_multi)
print_test

'DataFrame saved to pengujian_pegasus_multi.xlsx'

In [ ]:
# hybrid rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_multi["test"][i]["document"]
  result = f_hybrid_en(input, 2)

  summary_hybrid_multi.append(result)

file_name = 'pengujian_hybrid_multi.xlsx'

print_test = save_test_data(file_name, reference_summary_multi, summary_hybrid_multi)
print_test

'DataFrame saved to pengujian_hybrid_multi.xlsx'

PENGUJIAN PADA 50 ARTIKEL BILLSUM

In [ ]:
# import and load dataset
from datasets import load_dataset

dataset_billsum = load_dataset("billsum")

print(f"Features in billsum : {dataset_billsum['train'].column_names}")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Features in billsum : ['text', 'summary', 'title']


In [ ]:
summary_textrank_billsum = []
summary_pegasus_billsum = []
summary_hybrid_billsum = []
reference_summary_billsum = []

limit = 50

In [ ]:
for i in range (limit):
  # Inisial reference
  reference = dataset_billsum["test"][i]["summary"]
  reference_summary_billsum.append(reference)

In [ ]:
# textrank rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_billsum["test"][i]["text"]
  result = f_textrank_en(input, 2)

  summary_textrank_billsum.append(result)

file_name = 'pengujian_textrank_billsum.xlsx'

print_test = save_test_data(file_name, reference_summary_billsum, summary_textrank_billsum)
print_test

'DataFrame saved to pengujian_textrank_billsum.xlsx'

In [ ]:
# pegasus rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_billsum["test"][i]["text"]
  result = f_pegasus_en(input)

  summary_pegasus_billsum.append(result)

file_name = 'pengujian_pegasus_billsum.xlsx'

print_test = save_test_data(file_name, reference_summary_billsum, summary_pegasus_billsum)
print_test

'DataFrame saved to pengujian_pegasus_billsum.xlsx'

In [ ]:
# hybrid rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_billsum["test"][i]["text"]
  result = f_hybrid_en(input, 2)

  summary_hybrid_billsum.append(result)

file_name = 'pengujian_hybrid_billsum.xlsx'

print_test = save_test_data(file_name, reference_summary_billsum, summary_hybrid_billsum)
print_test

'DataFrame saved to pengujian_hybrid_billsum.xlsx'